# Using LangChain with GPT4All and Custom Prompts

## References

1. [LangChain docs on using the GPT4All library](https://python.langchain.com/docs/integrations/llms/gpt4all)

## Code

### Import libraries

In [1]:
from langchain.llms import GPT4All
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

### Setup

In [2]:
local_path = r"C:\Users\deshi\Code\gpt4all-models\mistral-7b-instruct-v0.1.Q4_0.gguf"
# local_path = r"C:\Users\deshi\Code\gpt4all-models\orca-mini-3b-gguf2-q4_0.gguf"

In [3]:
# Callbacks support token-wise streaming
callbacks = [StreamingStdOutCallbackHandler()]

# Verbose is required to pass to the callback manager
llm = GPT4All(
    model=local_path, 
    callbacks=callbacks, 
    verbose=True,
    max_tokens=512,
    n_batch=4
)

# If you want to use a custom model add the backend parameter
# Check https://docs.gpt4all.io/gpt4all_python.html for supported backends
# llm = GPT4All(model=local_path, backend="gptj", callbacks=callbacks, verbose=True)

### Standard template

In [4]:
template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate(template=template, input_variables=["question"])

In [5]:
llmChain = LLMChain(prompt=prompt, llm=llm)

### Regular Inference

In [6]:
question = """
Write a program to calculate the nth fibonacci number given a user input integer n. Also, print out all the n fibonacci numbers leading upto n in a list. Format the output in markdown format so it is easy to export to a markdown document.
"""

llmChain.run(question)

 

1. We need to write a program that calculates the nth Fibonacci number.
2. To do this, we can use a loop to generate the previous two numbers in the sequence and add them together until we reach the desired value of n.
3. We can then print out the result using the built-in `print()` function.
4. Finally, we need to format the output so that it is easy to export to a markdown document. 

Here's the code:

```python
n = int(input("Enter an integer: "))
result = 0
previous_two = 1
while result < n:
 next_two = previous_two + previous_two
 result += next_two
 previous_two = next_two
 print("The nth Fibonacci number is:", result)
```

Now let's explain the code step by step. 

1. We start by taking an integer input from the user using the `int()` function and storing it in a variable called `n`.
2. We initialize two variables, `result` and `previous

' \n\n1. We need to write a program that calculates the nth Fibonacci number.\n2. To do this, we can use a loop to generate the previous two numbers in the sequence and add them together until we reach the desired value of n.\n3. We can then print out the result using the built-in `print()` function.\n4. Finally, we need to format the output so that it is easy to export to a markdown document. \n\nHere\'s the code:\n\n```python\nn = int(input("Enter an integer: "))\nresult = 0\nprevious_two = 1\nwhile result < n:\n next_two = previous_two + previous_two\n result += next_two\n previous_two = next_two\n print("The nth Fibonacci number is:", result)\n```\n\nNow let\'s explain the code step by step. \n\n1. We start by taking an integer input from the user using the `int()` function and storing it in a variable called `n`.\n2. We initialize two variables, `result` and `previous'

### Function Documentation Inference with Standard Template

In [7]:
testFunction = """
n = int(input("Enter an integer: "))
result = 0
previous_two = 1
while result < n:
    next_two = previous_two + previous_two
    result += next_two
    previous_two = next_two
    print("The nth Fibonacci number is:", result)
"""

question = f"""
Here's my function in Python:\

{testFunction}\

Given the definition of a function in any programming language (particularly Python and C++), please generate it's stand-alone documentation. I want it complete with fields like function name, function arguments and return values as well as a detailed explanation of how the function logic works line-by-line. Make it concise and informative to put the documentation into a project documentation file.
"""

In [8]:
llmChain.run(question)

 

1. Define the function: 
```python
def fibonacci(n):
 result = 0
 previous_two = 1
 while result < n:
 next_two = previous_two + previous_two
 result += next_two
 previous_two = next_two
 print("The nth Fibonacci number is:", result)
```

2. Call the function and pass an integer argument: 
```python
fibonacci(5) # Output: The 5th Fibonacci number is: 13
```

3. Explain how the function works step by step:
- `result = 0` initializes the variable `result` to 0.
- `previous_two = 1` sets the value of `previous_two` to 1.
- `while result < n` checks if the current value of `result` is less than the input parameter `n`.
- `next_two = previous_two + previous_two` calculates the next two numbers in the sequence by adding the previous two values together.
- `result

' \n\n1. Define the function: \n```python\ndef fibonacci(n):\n result = 0\n previous_two = 1\n while result < n:\n next_two = previous_two + previous_two\n result += next_two\n previous_two = next_two\n print("The nth Fibonacci number is:", result)\n```\n\n2. Call the function and pass an integer argument: \n```python\nfibonacci(5) # Output: The 5th Fibonacci number is: 13\n```\n\n3. Explain how the function works step by step:\n- `result = 0` initializes the variable `result` to 0.\n- `previous_two = 1` sets the value of `previous_two` to 1.\n- `while result < n` checks if the current value of `result` is less than the input parameter `n`.\n- `next_two = previous_two + previous_two` calculates the next two numbers in the sequence by adding the previous two values together.\n- `result'

### Custom Prompt Template

In [35]:
template = """
Given the definition of a function in any programming language, please generate it's documentation. The documentation needs to have fields like function name, function arguments and return values as well as a detailed explanation of how the function logic works line-by-line.

This is a function in {language}:
{function}

The response needs to be structured as:

# Function:
The function definition and body here.
# Arguments:
List of arguments along with their types.
# Return values:
List of all the values returned by the function with their types.
# Explanation:
Line-by-line explanation of the function's logic and implementation.
# Use cases:
How the function may be used in an intended context.
"""

prompt = PromptTemplate(
    template=template,
    input_variables=["language", "function"],
)

In [36]:
llmChain = LLMChain(
    prompt=prompt, 
    llm=llm,
)

In [37]:
# Be specific about the language to avoid LLM hallucination
language = "Python"

print(prompt.format(language=language, function=testFunction))


Given the definition of a function in any programming language, please generate it's documentation. The documentation needs to have fields like function name, function arguments and return values as well as a detailed explanation of how the function logic works line-by-line.

This is a function in Python:

n = int(input("Enter an integer: "))
result = 0
previous_two = 1
while result < n:
    next_two = previous_two + previous_two
    result += next_two
    previous_two = next_two
    print("The nth Fibonacci number is:", result)


The response needs to be structured as:

# Function:
The function definition and body here.
# Arguments:
List of arguments along with their types.
# Return values:
List of all the values returned by the function with their types.
# Explanation:
Line-by-line explanation of the function's logic and implementation.
# Use cases:
How the function may be used in an intended context.



In [38]:
llmChain.run({"language": language, "function": testFunction})


The given Python code is a Fibonacci sequence function that takes an integer input and returns the nth Fibonacci number. The function works by initializing two variables, previous_two and next_two, to 1 and 0 respectively. Then it enters a while loop that continues as long as the result variable is less than n. In each iteration of the loop, it calculates the next two numbers in the sequence using the formula: previous_two + previous_two. It then adds these numbers to the result variable and updates the previous_two variable to be those two numbers. Finally, it prints out the value of the result variable as the nth Fibonacci number.

'\nThe given Python code is a Fibonacci sequence function that takes an integer input and returns the nth Fibonacci number. The function works by initializing two variables, previous_two and next_two, to 1 and 0 respectively. Then it enters a while loop that continues as long as the result variable is less than n. In each iteration of the loop, it calculates the next two numbers in the sequence using the formula: previous_two + previous_two. It then adds these numbers to the result variable and updates the previous_two variable to be those two numbers. Finally, it prints out the value of the result variable as the nth Fibonacci number.'